In [13]:
import pandas as pd
import numpy as np
import pickle
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix, csc_matrix
from pandas.api.types import CategoricalDtype
from IPython.display import clear_output
import timeit
import warnings

# Ineraction Matrix
----

In [14]:
data = np.array([[1,1,2,3,1,2,3,3,3,3],['A','A','B','B','C','D','C','C','C','D']]).T

In [15]:
df = pd.DataFrame(data, columns = ['user_id','product_name'])
df

,user_id,product_name
0,1,A
1,1,A
2,2,B
3,3,B
4,1,C
5,2,D
6,3,C
7,3,C
8,3,C
9,3,D


In [16]:
# create interaction matrix
interaction_matrix_count = df.pivot_table(index='user_id', columns='product_name', aggfunc=len, fill_value=0)
interaction_matrix_count.head()

product_name,A,B,C,D
user_id,,,,
1,2,0,1,0
2,0,1,0,1
3,0,1,3,1


In [17]:
#interaction_matrix_count = df.pivot_table(index='user_id', columns='product_name', values=[1])

In [18]:
interaction_matrix_count.applymap(lambda x: int(x>0))

product_name,A,B,C,D
user_id,,,,
1,1,0,1,0
2,0,1,0,1
3,0,1,1,1


## Interaction Matrix as sparse Matrix
----

In [19]:
# df = pd.read_csv('freq.csv')

In [20]:
def interaction(df, val = None, recommender='item'):
    user_c = CategoricalDtype(sorted(df.user_id.unique()), ordered=True)
    product_name_c = CategoricalDtype(sorted(df.product_name.unique()), ordered=True)
    
    row = df.user_id.astype(user_c).cat.codes
    col = df.product_name.astype(product_name_c).cat.codes
    if val is None:
        val = [1]*len(col)
    if recommender == 'item':     
        sparse_matrix = csc_matrix((val,(row, col)), shape=(user_c.categories.size, product_name_c.categories.size))
    elif recommender == 'user':     
        sparse_matrix = csr_matrix((val,(row, col)), shape=(user_c.categories.size, product_name_c.categories.size))

    return sparse_matrix

In [21]:
sparse_matrix = interaction(df, recommender='item')
sparse_matrix.todense()

matrix([[2, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 1, 3, 1]], dtype=int32)

In [22]:
# binary sparse matrix:
sparse_binary = sparse_matrix
sparse_binary[sparse_binary.nonzero()] = 1
sparse_binary.todense()

matrix([[1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 1, 1, 1]], dtype=int32)

In [23]:
# turn dense matrix to dataframe
# dfs = pd.DataFrame.sparse.from_spmatrix(sparse_matrix, index=user_c.categories, columns=product_name_c.categories)

In [24]:
# extract of a product and its values for diffrent customers in interaction matrix. 
# dfs.iloc[:, 401].value_counts()

# Similarity Measures
----

## Cosine Similarity
Cosine similarity measures the similarity between two vectors. To achieve this it calculates the angle between the two vectors. With an intermediate angle of $0°$ (the two vectors point in the same direction) the similarity is $1$, ($cos(0) = 1$). For any other angle the value is less than one. In our example we consider products (columns of the interaction matrix) or users (rows of the interaction matrix) as vectors. In our formula are $i$ and $j$ items which each have a vector consisting of all their ratings $r$. Note: This function can be used for UBCF and IBCF.

$$
s(i,j) = \frac{r_{i}\cdot r_{j}}{\|r_{i}\|_{2}\cdot\|r_{j}\|_{2}}
$$

<span style='color:red'>TO DO: Add functionality for UBCF, Build decorator for estimator</span>

In [25]:
sparse_matrix = interaction(df, recommender='item')

In [26]:
def sim_cosine(df, recommender, time=False):
    """
    Calculates the cosine similarity between two given vectors
    :param df: sparse matrix with shape (user,item)
    :return s: similarity value between -1 and 1 (1 high correlation, 0 no correlation, -1 high negative correlation)
    """
    start = timeit.default_timer()
    
    if recommender == 'item':
    
        # initialize empty diagonal matrix
        length = df.shape[1]
        similarity_matrix = np.zeros((length,length), dtype=np.float32) # empty similarity matrix

        # precalculate normalized vector over whole df
        normalized_vectors = norm(df, axis=0)


        for i in np.arange(length):
            clear_output(wait=True)

            # cosine similarity calculation
            a = df[:,i]
            numerator = df.T.dot(a).todense().T # get the dotproduct for vector a and every other vector
            denominator = normalized_vectors*normalized_vectors[i]

            s = numerator/denominator
            similarity_matrix[i,:] = s

            # progress estimator:
            if time:
                percent_old = 0
                percent_new = int(100 * (i / length))

                stop = timeit.default_timer()

                expected_time = np.round(length* ((stop-start)/i) /60,2)
                print(f'Current progress: {percent_new}%')
                print(f'Current run tume: {np.round((stop-start)/60,2)} minutes')
                print(f'Expected run time: {expected_time} minutes')
                percent_old = percent_new
                
    elif recommender == 'user':
        
        # initialize empty diagonal matrix
        length = df.shape[0]
        similarity_matrix = np.zeros((length,length), dtype=np.float32) # empty similarity matrix

        # precalculate normalized vector over whole df
        normalized_vectors = norm(df, axis=1)


        for i in np.arange(length):
            clear_output(wait=True)

            # cosine similarity calculation
            a = df[i,:]
            numerator = df.dot(a.T).todense().T # get the dotproduct for vector a and every other vector
            denominator = normalized_vectors*normalized_vectors[i]

            s = numerator/denominator
            similarity_matrix[i,:] = s

            # progress estimator:
            if time:
                percent_old = 0
                percent_new = int(100 * (i / length))

                stop = timeit.default_timer()

                expected_time = np.round(length* ((stop-start)/i) /60,2)
                print(f'Current progress: {percent_new}%')
                print(f'Current run tume: {np.round((stop-start)/60,2)} minutes')
                print(f'Expected run time: {expected_time} minutes')
                percent_old = percent_new

    return similarity_matrix

In [27]:
sim_cosine(sparse_matrix, recommender='item', time=True)

Current progress: 75%
Current run tume: 0.0 minutes
Expected run time: 0.0 minutes


array([[1.        , 0.        , 0.31622776, 0.        ],
       [0.        , 1.        , 0.6708204 , 1.        ],
       [0.31622776, 0.6708204 , 1.        , 0.6708204 ],
       [0.        , 1.        , 0.6708204 , 1.        ]], dtype=float32)

# Pearson Correlation
----
<span style='color:red'>Not functioning at the moment. sim_pearson is still in the works.</span>
Person Correlation measures the statistical correlation between the ratings of two user's (or items). The formula takes as input two users $u$ and $v$ and calculates a similarity $s$ using the ratings $r$ for every product the two users have in common. It is possible to use pearson correlation to measure the similarity between items as well. But it is said that it yields **worse results** as a cosine similarity for example. The pearson correlation also suffers from problems if there are just a few ratings to compare. A threshold of a number of ratings per user could counteract this problem.

$$
s(u,v) = \frac{\sum_{i\in I_{u}\cap I_{v}}(r_{u,i}-\bar r_{u})(r_{v,i}-\bar r_{v})}{\sqrt{\sum_{i\in I_{u}\cap I_{v}}(r_{u,i}-\bar r_{u})^2} \sqrt{\sum_{i\in I_{u}\cap I_{v}}(r_{v,i}-\bar r_{v})^2}}
$$

In [28]:
sparse_matrix = interaction(df, recommender='user') # 'user' returns a csc-matrix (c=column)
sparse_matrix.todense()

matrix([[2, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 1, 3, 1]], dtype=int32)

In [36]:
def sim_pearson(df, recommender, time=False, threshold=False):
    """
    Calculates the pearson similarity between two given users
    :param df: sparse matrix shape (user, item)
    :param threshold: activate involvement of a threshold "multiplication" of the function
    :return s: similarity value between -1 and 1 (1 high correlation, 0 no correlation, -1 high negative correlation) or NaN if calculation is not possible
    """
    start = timeit.default_timer()
    
    if recommender == 'user':
        
        # initialize empty diagonal matrix
        length = df.shape[0] # length of user vector
        item_length = df.shape[1] # length of item vector
        similarity_matrix = np.zeros((length,length), dtype=np.float32) # empty similarity matrix
        
        mean_rating = [df[i,:].sum(axis=1)[0,0]/df[i,:].count_nonzero() for i in np.arange(3)] # get sum of all >0 values for every row and divide them by number of nonzero values.
        
        
        for i in np.arange(length):
            
            # calculate item overlap between user i and every other user (list of lists)
            overlap = [[index for index,v,u in zip(np.arange(item_length),df[:,i],df[:,x]) if v > 0 and u > 0 ] for x in np.arange(length)]

            numerator = sum([(df[overlap[]])])
            print(numerator)
            numerator = sum(a*b for a,b in zip([u[i] -  for i in intersection],[v[i] - v_mean_r for i in intersection]))
            denominator1 = np.sqrt(sum([(u[i] - u_mean_r)**2 for i in intersection]))
            denominator2 = np.sqrt(sum([(v[i] - v_mean_r)**2 for i in intersection]))

            s = numerator / (denominator1 * denominator2)
            if threshold:
                s = s * min(len(interesection)/50,1)


            clear_output(wait=True)
            # progress estimator:
            if time:
                percent_old = 0
                percent_new = int(100 * (i / length))

                stop = timeit.default_timer()

                expected_time = np.round(length* ((stop-start)/i) /60,2)
                print(f'Current progress: {percent_new}%')
                print(f'Current run tume: {np.round((stop-start)/60,2)} minutes')
                print(f'Expected run time: {expected_time} minutes')
                percent_old = percent_new

    if recommender == 'item':
        return warnings.warn("Similartiy Method 'pearson' does not support IBCF yet. Use parameter recommender='user' for UBCF")

    return s

SyntaxError: invalid syntax (<ipython-input-36-1e4cf54fab03>, line 27)

In [75]:
u = np.array(sparse_matrix[0,:].todense())[0]
v = np.array(sparse_matrix[1,:].todense())[0]
intersection = [k for k, i, j in zip(np.arange(u.shape[0]),u,v) if i > 0  and j > 0]
intersection

[]

## Similarity Matrix
----
The function `similarity()` creates a similarity matrix using a similarity function (for example cosine similarity (`sim_cosine`). 

In [76]:
def similarity(df, sim='cosine', recommender='item'):

    # precalculate normalized vectors if sim = cosine
    if sim == 'cosine':
        similarity_matrix = sim_cosine(df, recommender)
    elif sim == 'pearson':
        similarity_matrix = sim_pearson(df, recommender)
    
    return similarity_matrix

In [77]:
%%time
similarity_items = similarity(sparse_matrix, recommender = 'item')


Wall time: 8.98 ms


## Recommend
----

In [78]:
def product_names(df):
    """
    returns a list of products contained in the df. The index is the corresponding index number in the similarity matrix.
    """
    product = pd.DataFrame(sorted(df.product_name.unique()))

    return product

product_names(df)

,0
0,A
1,B
2,C
3,D


In [79]:
def recommend(matrix_name, item_id, nr_of_items):
  
    matrix = pickle.load(open(matrix_name, 'rb'))
    #Sets diagonal to zero (if we dont want to recomend the item the user has just bought)
    np.fill_diagonal(matrix, -2)
    
    #gets two list of item index and item similarity rating
    nr_of_rows = matrix.shape[0]
    index = np.zeros((nr_of_rows, nr_of_items))
    ratings = np.zeros((nr_of_rows, nr_of_items))
    for row in range(nr_of_rows):
        index[row,:] = matrix[row].argsort()[-nr_of_items:][::-1].tolist()
        ratings[row,:] = matrix[row, index[row,:].astype(int)]
        
    # print results
    print("Recommendation for {}:".format(item_id))
    for i in range(index.shape[1]):
        print("{}: {} with a similarity rating of {} ".format((i+1), int(index[item_id, i]), round(ratings[item_id, i], 3)))

    return 

In [80]:
# pd.read_csv('data.csv')  
item_similarity = recommend(matrix_name = 'rating_count_item_similarity.pkl', item_id = 2, nr_of_items = 15)

FileNotFoundError: [Errno 2] No such file or directory: 'rating_count_item_similarity.pkl'

## User Product Rating
----

#### Version 1

In order to get a user-item rating for the interaction matrix, we had to develop a rating function.

The following thoughts were made:
- We want to give sufficient weight to the first product purchase
    - We decided that the first purchase has a weight $\displaystyle \omega = \frac{1}{3}$
- We want to even more weight a reorder of a product (this assumes that the user liked the product)
    - To create the gap, we defined the minimum gap of $\displaystyle 2 \cdot \omega$
- In addition, we do not want to weight multiple reorders too much that users with e.g. 6 or 10 reorders differs not too much
    - To achieve this, we can take the square-root of the number of reorders since $\sqrt{x}\,\, |\,\, x \geq 0\,\,$ is concave
- The rating should be a number between 0 and 1

Therefore the following formula has been developed:

$\displaystyle{ rating(o, o_{tot}) =
  \begin{cases}
    0            & \quad \text{if } o \text{ is } 0\\
    \omega       & \quad \text{if } o \text{ is }1 \land o_{tot} > 1\\
    2 \cdot \omega + (1 - 2 \cdot \omega) \cdot \sqrt{\frac{o}{o_{tot}}}  & \quad \text{if } o \text{ is } \geq 1 \lor \left( o \text{ is }1 \land o_{tot} \text{ is } 1 \right)
  \end{cases}}$
    
- where $o$ is the number of orders of the specified product $p$ of the user and $o_{tot}$ is the number of orders of the user.
- $\omega$ must be well defined $0 < \omega < 0.5$, optimally somewhere in the middle

This approach does strongly weight the first reorder and neither takes the number of orders of a user nor the aisles into account.

In [ ]:
omega = 0.35
gamma = 10 # number of orders of the user
for o in range(gamma + 1):
    if o == 0:
        x = 0
    elif o == 1 and gamma > 1:
        x = omega
    else:
        x = 2*omega + (1-2*omega) * np.sqrt(o/gamma)
    print(x)

#### Version 2

In order to get a user-item rating for the interaction matrix, we had to develop a rating function.

The following thoughts were made:
- We want to give sufficient weight to the first product purchase
    - We decided that the first purchase has a weight $\displaystyle \omega = \frac{1}{3}$
    
- Our main weight should be the frequency of a product being in an order.
    - To achieve this, we can take the square-root of the number of orders containing a product  $o$ divided by the total number of orders by the customer $o_{tot}$: $\sqrt{\frac{o}{o_{tot}}}$
    
- Because there are a lot of customers with a low number of orders and related to that a specific uncertainty, we want to weaken the ratings for these customers.
    - To achieve this, we can take the square-root of the total amount of orders for each customer $o_{tot}$ divided by a specific treshold value $m$: $\sqrt{\frac{o_{tot}}{m}}\,\, |\,\, o_{tot} < m\,\,$

- The rating should be a number between 0 and 1

Therefore the following formula has been developed:

$\displaystyle{ rating(o, o_{tot}) =
  \begin{cases}
    0            & \quad \text{if } o \text{ is } 0\\
    \omega       & \quad \text{if } o \text{ is }1 \\
    \omega + (1 - \omega) \cdot \sqrt{\frac{o}{o_{tot}}}  & \quad \text{if } o \text{ is } > 1 \land \left(  o_{tot} \geq m \right)\\
    \omega + (1 - \omega) \cdot \sqrt{\frac{o}{o_{tot}}} \cdot \sqrt{\frac{o_{tot}}{m}}  & \quad \text{if } o \text{ is } > 1 \land \left(  o_{tot} < m \right)\\
  \end{cases}}$
    
- where $o$ is the number of orders of the specified product and $o_{tot}$ is the total amount of orders from the corresponding customer $p$.

This approach takes the ratio of each product beeing ordered by a customer and weakens the rating, if to little orders are aviable.

In [88]:
omega = 1/3
o_tot = 10 # products in o
# o total o of customer
m = 10 # lowers ratings ir less than m orders

for o_tot in range(m+20):
    print("o_tot =", o_tot)
    for o in range(o_tot + 1):
        if o == 0:
            x = 0
        elif o == 1:
            x = omega
        else:
            if o_tot < m:
                w_freq = np.sqrt(o_tot / m)
            else:
                w_freq = 1

            w_prod = np.sqrt(o / o_tot)
            
            x = omega + (1-omega) * w_prod * w_freq

        print("x =", x)

o_tot = 0
x = 0
o_tot = 1
x = 0
x = 0.3333333333333333
o_tot = 2
x = 0
x = 0.3333333333333333
x = 0.6314757303333053
o_tot = 3
x = 0
x = 0.3333333333333333
x = 0.6314757303333054
x = 0.6984817050034441
o_tot = 4
x = 0
x = 0.3333333333333333
x = 0.6314757303333054
x = 0.6984817050034442
x = 0.7549703546891173
o_tot = 5
x = 0
x = 0.3333333333333333
x = 0.6314757303333054
x = 0.6984817050034442
x = 0.7549703546891173
x = 0.804737854124365
o_tot = 6
x = 0
x = 0.3333333333333333
x = 0.6314757303333053
x = 0.6984817050034442
x = 0.7549703546891173
x = 0.804737854124365
x = 0.8497311128276557
o_tot = 7
x = 0
x = 0.3333333333333333
x = 0.6314757303333054
x = 0.6984817050034441
x = 0.7549703546891173
x = 0.804737854124365
x = 0.8497311128276557
x = 0.8911066843560504
o_tot = 8
x = 0
x = 0.3333333333333333
x = 0.6314757303333053
x = 0.698481705003444
x = 0.7549703546891173
x = 0.804737854124365
x = 0.8497311128276557
x = 0.8911066843560504
x = 0.9296181273332773
o_tot = 9
x = 0
x = 0.33333333333

In [81]:
@np.vectorize
def rating(o, o_tot, m = 10, omega = 1/3):
    if o == 0:
        x = 0
    elif o == 1:
        x = omega
    else:
        if o_tot < m:
            w_freq = np.sqrt(o_tot / m)
        else:
            w_freq = 1
        w_prod = np.sqrt(o / o_tot)
        x = omega + (1-omega) * w_prod * w_freq
    return x

In [82]:
df_rating = df.copy()
df_rating['order_id'] = np.array([[1,2,3,4,1,3,4,5,6,6]]).T
df_rating

,user_id,product_name,order_id
0,1,A,1
1,1,A,2
2,2,B,3
3,3,B,4
4,1,C,1
5,2,D,3
6,3,C,4
7,3,C,5
8,3,C,6
9,3,D,6


In [83]:
df_o = df_rating.groupby(by = ['user_id', 'product_name']).count().rename(columns = {'order_id':'o'}).reset_index() #creates the number of orders per unique product
o_tot = df_rating.groupby(by = ['user_id'])['order_id'].nunique() #creates the total number of orders from a user
df_rating = df_o.join(o_tot, on = 'user_id').rename(columns = {'order_id':'o_tot'})
df_rating

,user_id,product_name,o,o_tot
0,1,A,2,2
1,1,C,1,2
2,2,B,1,1
3,2,D,1,1
4,3,B,1,3
5,3,C,3,3
6,3,D,1,3


In [84]:
o = df_rating.loc[:, 'o'].to_numpy()
o_tot = df_rating.loc[:, 'o_tot'].to_numpy()
df_rating['rating'] = rating(o, o_tot).T
df_rating

,user_id,product_name,o,o_tot,rating
0,1,A,2,2,0.631476
1,1,C,1,2,0.333333
2,2,B,1,1,0.333333
3,2,D,1,1,0.333333
4,3,B,1,3,0.333333
5,3,C,3,3,0.698482
6,3,D,1,3,0.333333


In [87]:
sparse_rating_matrix = interaction(df_rating, df_rating.loc[:, 'rating'], recommender='item')
sparse_rating = sparse_rating_matrix.todense()
sparse_rating

matrix([[0.63147573, 0.        , 0.33333333, 0.        ],
        [0.        , 0.33333333, 0.        , 0.33333333],
        [0.        , 0.33333333, 0.69848171, 0.33333333]])